In [1]:
import sys
sys.path.append('..')
from iltransr.models.il_smiles_temperature import*
viscosity_param_file = os.path.join(root_path,'iltransr/pre-trained params/viscosity_best.params')

[04:25:20] ../src/storage/storage.cc:199: Using Pooled (Naive) StorageManager for CPU
[04:25:22] ../src/storage/storage.cc:199: Using Pooled (Naive) StorageManager for GPU


In [2]:
dropout = 0.05
batch_size = 128

In [3]:
net = ILNet(dropout=dropout)
net.IL_encoder = model.encoder
net.IL_src_embed =  model.src_embed
net.hybridize()
print(net)

ILNet(
  (IL_src_embed): HybridSequential(
    (0): Embedding(72 -> 128, float32)
    (1): Dropout(p = 0.0, axes=())
  )
  (IL_encoder): TransformerEncoder(
    (layer_norm): LayerNorm(eps=1e-05, axis=-1, center=True, scale=True, in_channels=128)
    (transformer_cells): HybridSequential(
      (0): TransformerEncoderCell(
        (attention_cell): MultiHeadAttentionCell(
          (_base_cell): DotProductAttentionCell(
            (_dropout_layer): Dropout(p = 0, axes=())
          )
          (proj_query): Dense(128 -> 128, linear)
          (proj_key): Dense(128 -> 128, linear)
          (proj_value): Dense(128 -> 128, linear)
        )
        (proj): Dense(128 -> 128, linear)
        (ffn): PositionwiseFFN(
          (ffn_1): Dense(128 -> 1024, linear)
          (activation): Activation(relu)
          (ffn_2): Dense(1024 -> 128, linear)
          (layer_norm): LayerNorm(eps=1e-05, axis=-1, center=True, scale=True, in_channels=128)
        )
        (layer_norm): LayerNorm(eps=1e-

In [4]:
net.load_parameters(viscosity_param_file,ctx=ctx)
net.initialize(init=mx.init.Xavier(magnitude=magnitude), ctx=ctx)

In [5]:
import pandas as pd
viscosity_database = pd.read_excel('smiles.xlsx',sheet_name='To_be_calculated')

In [6]:
predict_IL_smiles = viscosity_database['SMILES'].map(canonical_smile).map(no_split).map(preprocess)
predict_T =viscosity_database['normalized_T_vis']
predict_dataset = gluon.data.SimpleDataset(gluon.data.ArrayDataset(predict_IL_smiles,predict_T))
predict_dataloader= get_predict_dataloader(predict_dataset,batch_size=batch_size)

In [7]:
predicted = predict(net, predict_dataloader,ctx)

[04:25:29] ../src/operator/cudnn_ops.cc:292: Auto-tuning cuDNN op, set MXNET_CUDNN_AUTOTUNE_DEFAULT to 0 to disable
[04:25:30] ../src/operator/cudnn_ops.cc:292: Auto-tuning cuDNN op, set MXNET_CUDNN_AUTOTUNE_DEFAULT to 0 to disable


In [8]:
df = pd.DataFrame(predicted)
df.to_csv('predicted_viscosity.csv')
print(df)

             0
0     7.912595
1     6.842937
2     7.128377
3     5.591047
4     5.442295
...        ...
5513  9.439482
5514  8.890400
5515  7.609926
5516  8.140711
5517  5.647591

[5518 rows x 1 columns]
